In [3]:
import re 
import numpy as np
import pandas as pd 

# Data Loading #

In [4]:
df = pd.read_csv('../data/minsk_flats.csv')

In [5]:
df = df.reset_index()
df = df[df['Год постройки'] < 2022]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11378 entries, 0 to 15459
Data columns (total 46 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   level_0                                 11378 non-null  int64  
 1   level_1                                 263 non-null    object 
 2   Адрес                                   11378 non-null  object 
 3   Направление                             91 non-null     object 
 4   Населенный пункт                        11378 non-null  object 
 5   Год постройки                           11378 non-null  float64
 6   Цена USD                                11378 non-null  object 
 7   Номер договора                          10791 non-null  object 
 8   Организация                             13 non-null     object 
 9   Ориентировочная стоимость эквивалентна  11378 non-null  object 
 10  Торг                                    1848 non-null   ob

In [6]:
features = ['Этаж / этажность', 'Тип дома', 'Высота потолков', 'Метро',
            'Район города', 'Цена USD', 'Год постройки', 'Площадь общая/жилая/кухня', 
            'Комнат всего/разд.', 'Балкон', 'Сан/узел']
df = df[features]
df.head()

,Этаж / этажность,Тип дома,Высота потолков,Метро,Район города,Цена USD,Год постройки,Площадь общая/жилая/кухня,Комнат всего/разд.,Балкон,Сан/узел
0,9 / 25,кирпичный,2.7 м,\n\nм. \n Академия...,"Советский районМикрорайон ""Я.Коласа-Рига, Некр...",300 000 USD,2016.0,93.2 / 64.7 / – м²,3 / 2,лоджия застекленная,2 сан.узла
1,10 / 25,каркасно-блочный,2.7 м,\n\nм. \n Аэродром...,"Октябрьский районМикрорайон ""Минск Мир (Minsk ...",51 900 USD,2021.0,32.3 / 25.7 / – м²,Свободная планировка (1-комнатная),лоджия застекленная,совмещенный
2,2 / 5,каркасно-блочный,2.68 м,NaN,"Советский районМикрорайон ""Зеленый луг""",135 000 USD,2019.0,125.2 / 122.8 / – м²,3 / 0,лоджия застекленная,2 сан.узла
3,7 / 12,каркасно-блочный,2.75 м,\n\nм. \n Московск...,"Первомайский районМикрорайон ""Макаенка""",132 000 USD,2020.0,86.1 / 69.8 / – м²,3 / 3,2 лоджии застекленные,раздельный
5,2 / 5,силикатные блоки,2.53 м,\n\nм. \n Тракторн...,"Партизанский районМикрорайон ""Багратиона, Менд...",56 500 USD,1965.0,59.9 / 42.6 / – м²,3 / 3,балкон застекленный,раздельный


# Data Cleaning # 

In [7]:
loggias = ["лоджия застекленная", "лоджия", "лоджия из кухни застеклена", "лоджия из кухни", "лоджия из кухни застеклена + вагонка", "балкон и лоджия"]
balconies = ["балкон застекленный", "балкон", "балкон застекленный + вагонка"]
two_balconies = ["2 лоджии застекленные", "2 лоджии", "2 балкона", "2 балкона застекленные", "балкон+терраса", "3 лоджии застекленные", 
                 "балкон+терраса", "2 лоджии застекленные + вагонка", "3 лоджии", "3 балкона", "2 балкона застекленные + вагонка", 
                 "2 лоджии 1 застекленная", "3 балкона застекленных", "2 балкона 1 застекленный"]

house_type_map = {'каркасно-блочный': 'frame-block', 'панельный': 'panel', 'кирпичный': 'brick', 
                  'монолитный': 'monolithic', 'блок-комнаты': 'block-rooms', 'силикатные блоки': 'silicate'}

map_bathroom = {'раздельный': 'separate', 'совмещенный': 'combined', '2 сан.узла': '2', 
                '3 сан.узла': 'more than 2', '4 сан.узла': 'more than 2'}

map_district = {'Октябрьский': 'Oktyabrsky', 'Центральный': 'Tsentralny', 'Московский': 'Moskovsky', 
                  'Фрунзенский': 'Frunzensky', 'Первомайский': 'Pervomaisky', 'Советский': 'Sovetsky', 
                  'Ленинский': 'Leninsky', 'Заводской':'Zavodskoy', 'Партизанский':'Partizansky'}

In [8]:
def get_floor(x):
    split = str(x).split('/')
    return int(split[0].strip()) if len(split) > 1 else np.nan

def get_num_of_storeys(x):
    split = str(x).split('/')
    return int(split[1].strip()) if len(split) > 1 else np.nan

def get_ceiling_height(x):
    x = re.findall(r"[-+]?\d*\.\d+|\d+", str(x))   
    return float(x[0]) if x else np.nan 

def get_target_price(x):
    x = re.findall(r'\b\d+\b', str(x))
    return float(x[0])*1000 if x else np.nan

def get_total_area(x):
    areas = str(x).split('/')
    areas += [np.nan]*(3-len(areas))    
    return float(areas[0].strip()) if str(areas[0].strip())[0].isnumeric() else np.nan

def get_living_area(x):
    areas = str(x).split('/')
    areas += [np.nan]*(3-len(areas))    
    return float(areas[1].strip()) if str(areas[1].strip())[0].isnumeric() else np.nan

def get_kitchen_area(x):
    areas = str(x).split('/')
    areas += [np.nan]*(3-len(areas))    
    return float(areas[2].split()[0]) if str(areas[2].split()[0].strip())[0].isnumeric() else np.nan

def map_balcony(x):
    if x == 'нет':
        return 'Without' 
    elif x in loggias:
        return 'Loggia'
    elif x in balconies:
        return 'Balcony'
    elif x in two_balconies:
        return 'Two balconies'
    else: 
        return np.nan

In [9]:
df = df.assign(
    floor = df['Этаж / этажность'].apply(get_floor),
    number_of_storeys = df['Этаж / этажность'].apply(get_num_of_storeys),
    ceiling_height = df['Высота потолков'].apply(get_ceiling_height),
    target_price = df['Цена USD'].apply(get_target_price),
    total_area = df['Площадь общая/жилая/кухня'].apply(get_total_area),
    living_area = df['Площадь общая/жилая/кухня'].apply(get_living_area),
    kitchen_area = df['Площадь общая/жилая/кухня'].apply(get_kitchen_area),
    district = df['Район города'].apply(lambda x: str(x).split()[0]).map(map_district),
    near_the_subway = df['Метро'].apply(lambda x: 'Yes' if x is not np.nan else 'No'),
    number_of_rooms = df['Комнат всего/разд.'].apply(lambda x: re.findall(r'\d+', x)[0]),
    house_type = df['Тип дома'].map(house_type_map),
    bathroom = df['Сан/узел'].map(map_bathroom),
    balcony = df['Балкон'].map(map_balcony),
    year_built = df['Год постройки']
)

In [10]:
df = df.drop(features, axis=1)
df.head()

,floor,number_of_storeys,ceiling_height,target_price,total_area,living_area,kitchen_area,district,near_the_subway,number_of_rooms,house_type,bathroom,balcony,year_built
0,9.0,25.0,2.70,300000.0,93.2,64.7,NaN,Sovetsky,Yes,3,brick,2,Loggia,2016.0
1,10.0,25.0,2.70,51000.0,32.3,25.7,NaN,Oktyabrsky,Yes,1,frame-block,combined,Loggia,2021.0
2,2.0,5.0,2.68,135000.0,125.2,122.8,NaN,Sovetsky,No,3,frame-block,2,Loggia,2019.0
3,7.0,12.0,2.75,132000.0,86.1,69.8,NaN,Pervomaisky,Yes,3,frame-block,separate,Two balconies,2020.0
5,2.0,5.0,2.53,56000.0,59.9,42.6,NaN,Partizansky,Yes,3,silicate,separate,Balcony,1965.0


# Saving #

In [12]:
df.to_csv('../data/dataset.csv', index=False)